### **Bài Tập 3: Gửi Email tự động bằng python**

**Mục tiêu:**
Tự động gửi email qua Gmail và nhận email từ hộp thư đến (inbox) sử dụng Python. Thông tin tài khoản (email, App Password), nội dung email, địa chỉ người nhận, và tiêu chí nhận email được lấy từ các tệp văn bản. Sử dụng thư viện smtplib, imaplib, và email để thực hiện yêu cầu.

---

### **Bước 1: Đọc thông tin tài khoản từ tệp văn bản**

**Ý tưởng:**
Để gửi và nhận email qua Gmail, chúng ta cần đăng nhập vào tài khoản Gmail bằng địa chỉ email và App Password. Thông tin này sẽ được lấy từ tệp văn bản chứa email và App Password.

**Hướng dẫn:**

1. **Đọc tài khoản từ tệp văn bản:**
    Đọc địa chỉ email và App Password từ tệp văn bản để sử dụng khi đăng nhập vào máy chủ SMTP (gửi) và IMAP (nhận).

2. **Mã lệnh Python:**

   ```python
    # Đọc thông tin tài khoản từ tệp văn bản
    with open('email_credentials.txt', 'r') as file:
        account_info = file.readlines()
        email = account_info[0].strip()  # Đọc email từ dòng đầu tiên
        app_password = account_info[1].strip()  # Đọc App Password từ dòng thứ hai
   ```

   **Giải thích:**

   * **open('email_credentials.txt', 'r')**: Mở tệp văn bản chứa thông tin tài khoản để đọc.
   * **readlines()**: Đọc tất cả các dòng trong tệp và lưu vào danh sách.
   * **strip()**: Loại bỏ khoảng trắng thừa và ký tự newline.

---

### **Bước 2: Đọc nội dung email, thông tin người nhận, và tiêu chí nhận email từ tệp văn bản**

**Ý tưởng:**
Đọc nội dung email gửi đi, địa chỉ người nhận, và tiêu chí lọc email nhận được (ví dụ: email từ một người gửi cụ thể) từ các tệp văn bản riêng biệt. Nội dung email gửi đi và email nhận được sẽ được xử lý để sử dụng trong các bước tiếp theo.

**Hướng dẫn:**

1. **Đọc nội dung bài viết từ tệp văn bản:**

   ```python
    # Đọc nội dung email gửi đi từ tệp văn bản
    with open('email_content.txt', 'r') as file:
        email_content = file.read()  # Đọc toàn bộ nội dung email
    
    # Đọc địa chỉ người nhận từ tệp văn bản
    with open('recipient.txt', 'r') as file:
        recipient_email = file.read().strip()  # Đọc địa chỉ email người nhận

    # Đọc tiêu chí nhận email từ tệp văn bản (ví dụ: email người gửi)
    with open('email_filter.txt', 'r') as file:
        filter_sender = file.read().strip()  # Đọc địa chỉ email người gửi để lọc
   ```

   **Giải thích:**

   * **read()**: Đọc toàn bộ nội dung trong tệp `email_content.txt` vào biến `email_content`.
   * **read().strip()**: Đọc nội dung tệp `recipient.txt` và `email_filter.txt`, loại bỏ khoảng trắng hoặc ký tự xuống dòng để lấy địa chỉ email người nhận và tiêu chí lọc (email người gửi).
   * **email_filter.txt**: Chứa địa chỉ email của người gửi mà bạn muốn lọc (ví dụ: sender@example.com).

---

### **Bước 3: Sử dụng smtplib để gửi email và imaplib để nhận email**

**Ý tưởng:**
Sử dụng smtplib để gửi email qua máy chủ SMTP của Gmail và imaplib để nhận email từ hộp thư đến. Email nhận được sẽ được lọc dựa trên tiêu chí (người gửi) từ tệp email_filter.txt, và nội dung email sẽ được in ra (hoặc lưu tùy theo nhu cầu).

**Hướng dẫn:**

1. **Sử dụng thư viện smtplib, imaplib, và email:**

    * Các thư viện này có sẵn trong Python, không cần cài đặt thêm.

2. **Gửi email qua Gmail**

   ```python
    import smtplib
    import imaplib
    import email
    from email.mime.text import MIMEText

    # Gửi email
    # Tạo nội dung email
    msg = MIMEText(email_content)
    msg['Subject'] = 'Email Tự Động'
    msg['From'] = email
    msg['To'] = recipient_email

    # Kết nối và gửi email qua máy chủ SMTP của Gmail
    with smtplib.SMTP('smtp.gmail.com', 587) as server:
        server.starttls()  # Kích hoạt mã hóa TLS
        server.login(email, app_password)  # Đăng nhập bằng email và App Password
        server.send_message(msg)  # Gửi email

    # Nhận email
    # Kết nối đến máy chủ IMAP của Gmail
    with imaplib.IMAP4_SSL('imap.gmail.com') as server:
        server.login(email, app_password)  # Đăng nhập bằng email và App Password
        server.select('INBOX')  # Chọn hộp thư đến

        # Tìm kiếm email từ người gửi cụ thể
        _, data = server.search(None, f'FROM "{filter_sender}"')
        for num in data[0].split():
            _, msg_data = server.fetch(num, '(RFC822)')
            email_msg = email.message_from_bytes(msg_data[0][1])
            subject = email_msg['subject']
            print(f"Tiêu đề email nhận được: {subject}")
            # Lấy nội dung email (chỉ văn bản thuần túy)
            if email_msg.is_multipart():
                for part in email_msg.walk():
                    if part.get_content_type() == 'text/plain':
                        print(f"Nội dung: {part.get_payload(decode=True).decode()}")
            else:
                print(f"Nội dung: {email_msg.get_payload(decode=True).decode()}")
            break  # Chỉ in email đầu tiên để đơn giản
   ```

   **Giải thích:**

    * **Gửi Email:**
        + **MIMEText(email_content):** Tạo nội dung email dạng văn bản thuần túy.
        + **msg['Subject'], msg['From'], msg['To']:** Thiết lập tiêu đề, người gửi, và người nhận.
        + **smtplib.SMTP('smtp.gmail.com', 587):** Kết nối đến máy chủ SMTP của Gmail.
        + **server.starttls():** Kích hoạt mã hóa TLS.
        + **server.login(email, app_password):** Đăng nhập bằng App Password.
        + **server.send_message(msg):** Gửi email.
    * **Nhận Email:**
        + **imaplib.IMAP4_SSL('imap.gmail.com'):** Kết nối đến máy chủ IMAP của Gmail qua SSL.
        + **server.login(email, app_password):** Đăng nhập bằng App Password.
        + **server.select('INBOX'):** Chọn hộp thư đến.
        + **server.search(None, f'FROM "{filter_sender}"'):** Tìm kiếm email từ người gửi được chỉ định trong `email_filter.txt`.
        + **server.fetch(num, '(RFC822)'):** Lấy toàn bộ nội dung email.
        + **email.message_from_bytes:** Phân tích email để lấy tiêu đề và nội dung.
        + **email_msg.is_multipart():** Kiểm tra email có nhiều phần (multipart) hay không để lấy nội dung văn bản thuần túy.
        + **break:** Chỉ xử lý email đầu tiên để giữ mã đơn giản, tương tự mức độ khó của bài gốc.